In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
%%time
statements_df = pd.read_csv('statements.csv')
statements_df

Wall time: 40.4 s


,source_item_id,edge_property_id,target_item_id
0,1,31,36906466
1,1,279,3695190
2,1,398,497745
3,1,398,1133705
4,1,398,1139177
...,...,...,...
141206848,77257484,59,9286
141206849,77257491,31,318
141206850,77257491,59,9286
141206851,77257493,31,318


In [3]:
page_df = pd.read_csv('page.csv')
page_dict = pd.Series(page_df.page_id.values,index=page_df.item_id).to_dict()
del(page_df)
page_dict[6199]

12

In [9]:
%%time
statements_df_page = statements_df.copy()
statements_df_page['source_page_id'] = [page_dict.get(i) for i in statements_df_page.source_item_id]
statements_df_page['target_page_id'] = [page_dict.get(i) for i in statements_df_page.target_item_id]
statements_df_page

Wall time: 2min 44s


,source_item_id,edge_property_id,target_item_id,source_page_id,target_page_id
0,1,31,36906466,31880.0,NaN
1,1,279,3695190,31880.0,50723558.0
2,1,398,497745,31880.0,6207679.0
3,1,398,1133705,31880.0,17365007.0
4,1,398,1139177,31880.0,435544.0
...,...,...,...,...,...
141206848,77257484,59,9286,NaN,152710.0
141206849,77257491,31,318,NaN,12558.0
141206850,77257491,59,9286,NaN,152710.0
141206851,77257493,31,318,NaN,12558.0


In [10]:
statements_df_page.drop(columns=['source_item_id', 'target_item_id'], inplace=True)
statements_df_page = statements_df_page.dropna()
statements_df_page = statements_df_page[['source_page_id', 'edge_property_id', 'target_page_id']]
statements_df_page

,source_page_id,edge_property_id,target_page_id
1,31880.0,279,50723558.0
2,31880.0,398,6207679.0
3,31880.0,398,17365007.0
4,31880.0,398,435544.0
5,31880.0,398,33196876.0
...,...,...,...
140846125,2368655.0,59,105200.0
140846126,2368655.0,361,962148.0
140853808,62057360.0,31,528282.0
140857402,62276323.0,31,682482.0


In [3]:
statements_np = statements_df.to_numpy(dtype=int)
del(statements_df)
del(statements_df_page)
statements_np

array([[       1,       31, 36906466],
       [       1,      279,  3695190],
       [       1,      398,   497745],
       ...,
       [77257491,       59,     9286],
       [77257493,       31,      318],
       [77257493,       59,     8913]])

In [ ]:
#RUN THIS IS YOU WANT A BI-DIRECTIONAL GRAPH
#statements_np = np.concatenate((statements_np, statements_np[:, [2, 1, 0]]), axis=0)

In [4]:
%%time
statements_np = statements_np[np.argsort(statements_np[:, 0])]
statements_np

Wall time: 6.74 s


array([[       1,       31, 36906466],
       [       1,     1343,   602358],
       [       1,     1419,  5457948],
       ...,
       [77257491,       59,     9286],
       [77257493,       31,      318],
       [77257493,       59,     8913]])

In [5]:
u, indices = np.unique(statements_np[:,0], return_index=True)

In [6]:
indices1 = np.append(indices, len(statements_np))
indices1[-10:]

array([141206835, 141206837, 141206839, 141206841, 141206843, 141206845,
       141206847, 141206849, 141206851, 141206853], dtype=int64)

In [ ]:
%%time
statement_dict = {j:statements_np[indices1[i]:indices1[i+1], :0:-1].tolist() for i, j in enumerate(u)}
del(statements_np)
print(len(statement_dict.items()))
statement_dict

In [ ]:
%%time
#takes ~1 min for page
with open("graph_page_uni.p", 'wb') as f:
    pickle.dump(statement_dict, f, protocol=pickle.HIGHEST_PROTOCOL)